# Исследование надежности заемщиков.

### Описание проекта

  Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
  Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Описание данных

Данные предоставлены в таблице формата CSV, где:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data.head(10))    #  обзор первых 10-и строк таблицы
data.info()    #  обзор общей информации о таблице

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Проверка данных на аномалии и исправления.

На первом шаге (при обзоре первых строк таблицы) видно, что данные в столбце `days_employed` заполнены некорректно - встречаются отрицательные числа. Причиной такой ошибки может являться неверное заполнение формы или ошибка со знаком в формуле расчета трудового стажа в днях.

Истправим эту неточность:

In [2]:
data.loc[data['days_employed'] < 0, 'days_employed'] = data['days_employed'].abs()
data.head()    #  проверка корректности работы логического условия

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Шаг 2.2 Заполнение пропусков

Из информации о данных видно, что в столбцах `days_employed` и `total_income` есть пропуски. Так как тип данных `float64`, то можно сделать вывод, что пропуски принимают значения `NaN`.

Посчитаем долю пропущенных значений в каждом из столбцов:

In [3]:
print('Доля пропущенных значений в столбце days_employed:', data['days_employed'].isna().sum() / len(data['days_employed']))
print('Доля пропущенных значений в столбце total_income:', data['total_income'].isna().sum() / len(data['total_income']))

Доля пропущенных значений в столбце days_employed: 0.10099883855981417
Доля пропущенных значений в столбце total_income: 0.10099883855981417


Причиной пропуска данных может быть нежелание клиента давать информацию о своих доходах и трудовом стаже.

Значения в столбцах `days_employed` и `total_income` -  общий трудовой стаж в днях и ежемесячный доход, то есть количественные переменные. Пропуски в таких переменных заполняют средним арифметическим или медианой. Однако среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. Поэтому медиана более объективная информация о ежемесячном доходе клиентов и общем трудовом стаже.

Заполним пропуски медианным значением:

In [4]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.info()    #  убедимся, что пропуски заполнены

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.3. Изменение типов данных.

Изменим вещественный тип данных в столбцах days_employed и total_income на целочисленный:

In [5]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')  

data.info()    #  убедимся, что тип данных в колонках days_employed и total_income изменен на int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Сначала найдем количество **явных** дубликатов и удалим их:

In [6]:
data.duplicated().sum()

54

In [7]:
data = data.drop_duplicates().reset_index(drop=True)
data.shape[0]    #  количество строк после удаления явных дубликатов 

21471

Переведем значения в столбцах `education` и `family_status` к нижнему регистру и проверим уникальные значения других столбцов:

In [8]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
# print(data['family_status'].unique())
print(data['gender'].unique())
#  print(data['income_type'].unique())
#  print(data['purpose'].sort_values().unique())
print(data['children'].unique())

['F' 'M' 'XNA']
[ 1  0  3  2 -1  4 20  5]


Удалим строку из датафрейма, в которой указан пол `XNA`:

In [9]:
print(data[data['gender'] == 'XNA'])
data = data.drop(labels = [10690],axis = 0).reset_index(drop=True)
print(data['gender'].unique())

       children  days_employed  dob_years            education  education_id  \
10690         0           2358         24  неоконченное высшее             2   

          family_status  family_status_id gender income_type  debt  \
10690  гражданский брак                 1    XNA   компаньон     0   

       total_income               purpose  
10690        203905  покупка недвижимости  
['F' 'M']


Исправление аномалий в столбце `children`:

In [10]:
data.loc[data['children'] < 0, 'children'] = data['children'].abs()
data.loc[data['children'] == 20, 'children'] = 2
print(data['children'].unique())

[1 0 3 2 4 5]


Узнаем количество дубликатов после обработки столбца `education` и удалим их:

In [11]:
data.duplicated().sum()

17

In [12]:
data = data.drop_duplicates().reset_index(drop=True)
data.shape[0]    #  убедимся, что строки-дубликаты удалены

21453

Для поиска неявных дубликатов можно сначала посмотреть уникальные значения конкретного столбца методом `unique()`. Если в списке уникальных значений встретятся одинаковые значения, записанные разным способом (с использованием заглавных и строчных букв, с использованием разных языков и т.д.), то их необходимо исправить некоторыми методами `pandas`. Например, изменить регистр на нижний можно методом `str.lower()`. После такой обработки столбцов *неявные* дубликаты станут *явными*, а считать их количество легко методом `duplicated()` в связке с методом `sum()`. Останется совершить последнее действие - удалить теперь уже *явные* дубликаты методом `drop_duplicates()` и перезаписать индексы методом `reset_index()` с параметром `drop=True`.

Появление таких дубликатов связано, скорее всего, с человеческим фактором.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим датафрейм `data_education` со столбцами `education_id` и `education`:

In [13]:
data_education = data[['education_id', 'education']]
data_education = data_education.drop_duplicates().reset_index(drop=True)
data_education

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Таким же образом создадим датафрейм `data_family_status` со столбцами `family_status_id` и `family_status` :

In [14]:
data_family_status = data[['family_status_id', 'family_status']]
data_family_status = data_family_status.drop_duplicates().reset_index(drop=True)
data_family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [15]:
del data['education']    #  удаление из датафрейма столбца education
del data['family_status']    #  удаление из датафрейма столбца family_status
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

In [16]:
def total_income_category(total_income):  #  создание функции для категоризации дохода
    if 0 <= total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    return 'A'
data['total_income_category'] = data['total_income'].apply(total_income_category)    #  создание столбца с категорией
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [17]:
def purpose_category(purpose):    #  функция для категоризации целей кредита
    if 'образован' in purpose:
        return 'получение образования'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жиль' or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'образован' in purpose:
        return 'получение образования'
data['purpose_category'] = data['purpose'].apply(purpose_category)    #  создание столбца с категорией кредита
display(data.head(10))
print(data['purpose_category'].isna().sum())    #  проверка на наличие пустых строк в столбце purpose_category

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


0


### Ответы на вопросы.

Прежде, чем ответить на вопросы, подготовим сводные таблицы для каждого датафрейма.

1) сводная таблица для зависимости между количеством детей и возвратом кредита в срок:

In [18]:
data_children_pivot = data.pivot_table(index='children', columns='debt', values='dob_years', aggfunc='count')
data_children_pivot = data_children_pivot.fillna(0)  #  на всякий случай
    #  для наглядности добавим столбец с процентом клиентов, выплативших кредит в срок:
data_children_pivot['percent'] = (data_children_pivot[1] / (data_children_pivot[0] + data_children_pivot[1]) * 100).round().astype('int')
data_children_pivot['percent'] = data_children_pivot['percent'].astype(str) + '%'   #  форматирование значения для наглядности

2) сводная таблица для зависимости между семейным положением и возвратом кредита в срок: 

In [19]:
data_family_status_pivot = data.pivot_table(index='family_status_id', columns='debt', values='dob_years', aggfunc='count')
data_family_status_pivot = data_family_status_pivot.fillna(0)
data_family_status_pivot['percent'] = (data_family_status_pivot[1] / (data_family_status_pivot[0] + data_family_status_pivot[1]) * 100).round().astype('int')
data_family_status_pivot['percent'] = data_family_status_pivot['percent'].astype(str) + '%'
data_family_status_pivot = data_family_status.merge(data_family_status_pivot, on='family_status_id', how='left')
del data_family_status_pivot['family_status_id']
#  data_family_status_pivot

3) сводная таблица для зависимости между уровнем дохода и возвратом кредита в срок:

In [20]:
data_total_income_pivot = data.pivot_table(index='total_income_category', columns='debt', values='dob_years', aggfunc='count')
data_total_income_pivot = data_total_income_pivot.fillna(0)
data_total_income_pivot['percent'] = (data_total_income_pivot[1] / (data_total_income_pivot[0] + data_total_income_pivot[1]) * 100).round().astype('int')
data_total_income_pivot['percent'] = data_total_income_pivot['percent'].astype(str) + '%'

4) сводная таблица для зависимости между целями кредита и его возврата в срок: 

In [21]:
data_purpose_pivot = data.pivot_table(index='purpose_category', columns='debt', values='dob_years', aggfunc='count')
data_purpose_pivot = data_purpose_pivot.fillna(0)
data_purpose_pivot['percent'] = (data_purpose_pivot[1] / (data_purpose_pivot[0] + data_purpose_pivot[1]) * 100).round().astype('int')
data_purpose_pivot['percent'] = data_purpose_pivot['percent'].astype(str) + '%'

##### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Посмотрим на сводную таблицу:

In [22]:
data_children_pivot

debt,0,1,percent
children,,,
0,13027.0,1063.0,8%
1,4410.0,445.0,9%
2,1926.0,202.0,9%
3,303.0,27.0,8%
4,37.0,4.0,10%
5,9.0,0.0,0%


***Вывод :*** Из сводной таблицы видно, что клиенты, у которых либо нет детей, либо имеется ровно три ребенка, в 8% случаев не успевают закрыть кредит в срок. Клиенты с пятью детьми *всегда* погашают кредит в срок, в то время, как те, у кого один или два ребенка не закрывают кредит в срок в 9% случаев. Однако количество клиентов, обратившихся в банк для получения кредита сильно снижается с увеличением количества детей в семье, поэтому принимать такую статистику во внимание не следует.

##### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посмотрим на сводную таблицу:

In [23]:
data_family_status_pivot

,family_status,0,1,percent
0,женат / замужем,11408,931,8%
1,гражданский брак,3762,388,9%
2,вдовец / вдова,896,63,7%
3,в разводе,1110,85,7%
4,не женат / не замужем,2536,274,10%


***Вывод :*** Из сводной таблицы видно, что клиенты, у которых семейный статус не женат(не замужем), реже остальных возвращают кредит в срок. А клиенты, у котороых семейный статус вдовец(вдова) или в разводе - чаще, однако, в целом, такие клиенты в меньшинстве. А вот те, кто женат(замужем), и обращаются в банк за кредитом чаще остальных, и в должниках остаются редко - всего в 8%.

##### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Посмотрим на сводную таблицу:

In [24]:
data_total_income_pivot

debt,0,1,percent
total_income_category,,,
A,23,2,8%
B,4684,356,7%
C,14656,1360,8%
D,329,21,6%
E,20,2,9%


***Вывод :*** Судя по сводной таблице, клиенты с уровнем дохода `D` (от 30 001 р. до 50 000 р.) не успевают выплатить кредит в срок реже остальных - всего в 6%. Так же можно заметить, что чаще всего кредит берут клиенты с доходом из категории `C` - от 50 001 р. до 200 000 р., однако выплатить в срок успевают всего в 8% случаев. И реже всего с доходом из категории `А` (свыше 1 000 001 р.) и `Е` (до 30 000 р.), но имеют задолжности в 8% и 9% случаев соответственно. Клиетны с уровнем дохода `B` (от 200 001 р. до 1 000 000 р.) берут кредит достаточно часто, кроме того процент тех, кто не успел закрыть кредит вовремя достаточно мал - всего 7 %.

##### Как разные цели кредита влияют на его возврат в срок?

In [25]:
data_purpose_pivot

debt,0,1,percent
purpose_category,,,
операции с автомобилем,3903,403,9%
операции с недвижимостью,10028,782,7%
получение образования,3643,370,9%
проведение свадьбы,2138,186,8%


***Вывод :*** Судя по сводной таблице, клиенты, обратившиеся в банк с целью получения кредита на операции с недвижимостью в большинстве и не закрывают кредит в срок реже остальных - в 7%. А вот те клиенты, которые берут кредит на операции с автомобилем или получение образования - чаще (в 9%). Меньше всего тех, кто брал кредит на проведение свадьбы, однако не успевают закрыть его в 8% случаев.

## Общий вывод:

В ходе исследования была выявлена зависимость между выплатой кредита в срок и такими факторами, как:
* количество детей;
* семейное положение;
* уровень дохода;
* цель кредита.

Для наглядности, результаты исследования были представлены в сводных таблицах с процентным соотношением клиентов, не выплативших кредит в срок, ко всем клиентам из данной категории. По результатам исследования можно сформулировать следущие выводы:
1) самыми надежными заемщиками оказались те, кто не имеет детей вовсе или имеет ровно три ребенка (хотя последние достатчно редко обращаются в банк для получения кредита), а самыми ненадежными те, кто имеет одного, двух или четырех детей;

2) вдовцы (вдовы) и те, кто в разводе гораздо чаще выплачивают кредит в срок, нежели те, кто не женат(не замужем);

3) клиенты с уровнем дохода от 30 001 р. до 50 000 р. не успевают выплатить кредит в срок реже остальных - всего в 6%, в то время, как клиенты с уровнем дохода до 30 000 р. чаще остальных - в 9% случаев;

4) самыми надежными заемщиками оказались те, кто берет кредит на операции с недвижимостью, а самыми ненадежными те, кто берет кредит на операции с автомобилем или на получение образования.

***Рекомендации :***

По результатам исследования можно дать следущие рекомендации:

1) обратить внимание на такие категории заемщиков, как:
  * клиенты, не имеющие детей (или имеющие ровно троих детей);
  * вдовцы (вдовы) и те, кто в разводе;
  * клиенты с уровнем дохода от 30 001 р. до 50 000 р.;
  * клиенты, которые берут кредит на операции с недвижимостью;
  
*эти категории самые надежные.*

2) стоит реже одобрять кредит следующим категориям заемщиков:
 * клиенты, имеющие четырех детей;
 * клиенты, находящиеся в статусе не женат (не замужем);
 * клиенты с уровнем дохода до 30 000 р.;
 * клиенты, которые берут кредит на операции с автомобилем или на получение образования;
 
*эти категории самые ненадежные.*